In [1]:
import pandas as pd
import numpy as np
import glob
import re

In [131]:
raw_dir = '../data/raw'
interim_dir = '../data/interim/'

In [3]:
source_files = pd.DataFrame({'file_path' : sorted(glob.glob(f'{raw_dir}/*.xlsx'))})

In [4]:
source_files['variable_type'] = \
    source_files['file_path'] \
        .replace('../data/raw/[0-9][a-z] - ', '', regex=True) \
            .replace('.xlsx', '', regex=True).replace('\W{1,}', '_', regex=True) \
                .str.lower()

In [102]:
dfs = {}
for i in source_files.index:
    dfs[source_files.loc[i,'variable_type']] =  pd.read_excel(source_files.loc[i,'file_path'])

/Users/brentbaker/opt/miniconda3/envs/coffee_market_analysis/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/brentbaker/opt/miniconda3/envs/coffee_market_analysis/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [103]:
dfs.keys()

dict_keys(['total_production', 'domestic_consumption', 'gross_opening_stocks', 'exports_crop_year', 'exports_calendar_year', 'imports', 're_exports', 'prices_paid_to_growers', 'retail_prices', 'indicator_prices', 'non_member_imports', 'non_member_re_exports'])

In [104]:
x = dfs['total_production']
x.columns = x.iloc[2, :]
x = x.iloc[3:66, 0:32]
x = x.dropna(how='all')
x = x.drop(columns=x.columns[1])
x.rename(columns={'Crop year' : 'country'}, inplace=True)
x = x.melt(id_vars='country', var_name = 'crop_year', value_name='production_1k_bags')
x['country'] = x['country'].str.strip()
x['production_1k_bags'] = x['production_1k_bags'].astype('float', errors='ignore')
x['production_kg'] = x['production_1k_bags'] * 1000 * 60
x['production_lb'] = x['production_kg'] * 2.20462262185
harvest_groups = ['April group', 'July group', 'October group']
totals = ['Total']
x['harvest_group'] = np.where(x['country'].isin(harvest_groups), x['country'], np.NaN)
x['harvest_group'] = x['harvest_group'].ffill()
hg_rows = x.iloc[:,0].isin(harvest_groups)
hg_df = x.loc[hg_rows]
tot_rows = x.iloc[:,0].isin(totals)
tot_df = x.loc[tot_rows]
x = x.loc[~(hg_rows|tot_rows)]
hg_df.drop(columns='country', inplace=True)
tot_df.drop(columns=['country', 'harvest_group'], inplace=True)
x = x.merge(hg_df, how = 'left', on = ['crop_year', 'harvest_group'], suffixes=['', '_hg'])
x = x.merge(tot_df, how = 'left', on = ['crop_year'], suffixes=['', '_total'])
x = x[['harvest_group','country','crop_year','production_1k_bags','production_kg','production_lb','production_1k_bags_hg',
'production_kg_hg','production_lb_hg','production_1k_bags_total','production_kg_total','production_lb_total']]
dfs['total_production'] = x

In [105]:
x = dfs['domestic_consumption']
x.columns = x.iloc[2, :]
x = x.iloc[3:66, 0:32]
x = x.dropna(how='all')
x = x.drop(columns=x.columns[1])
x.rename(columns={'Crop year' : 'country'}, inplace=True)
x = x.melt(id_vars='country', var_name = 'crop_year', value_name='consumption_1k_bags')
x['country'] = x['country'].str.strip()
x['consumption_1k_bags'] = x['consumption_1k_bags'].astype('float', errors='ignore')
x['consumption_kg'] = x['consumption_1k_bags'] * 1000 * 60
x['consumption_lb'] = x['consumption_kg'] * 2.20462262185
harvest_groups = ['April group', 'July group', 'October group']
totals = ['Total']
x['harvest_group'] = np.where(x['country'].isin(harvest_groups), x['country'], np.NaN)
x['harvest_group'] = x['harvest_group'].ffill()
hg_rows = x.iloc[:,0].isin(harvest_groups)
hg_df = x.loc[hg_rows]
tot_rows = x.iloc[:,0].isin(totals)
tot_df = x.loc[tot_rows]
x = x.loc[~(hg_rows|tot_rows)]
hg_df.drop(columns='country', inplace=True)
tot_df.drop(columns=['country', 'harvest_group'], inplace=True)
x = x.merge(hg_df, how = 'left', on = ['crop_year', 'harvest_group'], suffixes=['', '_hg'])
x = x.merge(tot_df, how = 'left', on = ['crop_year'], suffixes=['', '_total'])
x = x[['harvest_group', 'country', 'crop_year', 'consumption_1k_bags', 'consumption_kg', 'consumption_lb', 'consumption_1k_bags_hg', 'consumption_kg_hg', 
'consumption_lb_hg', 'consumption_1k_bags_total', 'consumption_kg_total', 'consumption_lb_total']]
dfs['domestic_consumption'] = x

In [106]:
x = dfs['gross_opening_stocks']
x.columns = x.iloc[2, :]
x = x.iloc[3:66, 0:32]
x = x.dropna(how='all')
x = x.drop(columns=x.columns[1])
x.rename(columns={'Crop years' : 'country'}, inplace=True)
x = x.melt(id_vars='country', var_name = 'crop_year', value_name='openstock_1k_bags')
x['country'] = x['country'].str.strip()
x['openstock_1k_bags'] = x['openstock_1k_bags'].astype('float', errors='ignore')
x['openstock_kg'] = x['openstock_1k_bags'] * 1000 * 60
x['openstock_lb'] = x['openstock_kg'] * 2.20462262185
harvest_groups = ['April group', 'July group', 'October group']
totals = ['Total']
x['harvest_group'] = np.where(x['country'].isin(harvest_groups), x['country'], np.NaN)
x['harvest_group'] = x['harvest_group'].ffill()
hg_rows = x.iloc[:,0].isin(harvest_groups)
hg_df = x.loc[hg_rows]
tot_rows = x.iloc[:,0].isin(totals)
tot_df = x.loc[tot_rows]
x = x.loc[~(hg_rows|tot_rows)]
hg_df.drop(columns='country', inplace=True)
tot_df.drop(columns=['country', 'harvest_group'], inplace=True)
x = x.merge(hg_df, how = 'left', on = ['crop_year', 'harvest_group'], suffixes=['', '_hg'])
x = x.merge(tot_df, how = 'left', on = ['crop_year'], suffixes=['', '_total'])
x = x[['harvest_group', 'country', 'crop_year', 'openstock_1k_bags', 'openstock_kg', 'openstock_lb', 'openstock_1k_bags_hg', 'openstock_kg_hg', 
'openstock_lb_hg', 'openstock_1k_bags_total', 'openstock_kg_total', 'openstock_lb_total']]
dfs['gross_opening_stocks'] = x

In [107]:
x = dfs['exports_crop_year']
x.columns = x.iloc[2, :]
x = x.iloc[3:66, 0:32]
x = x.dropna(how='all')
x = x.drop(columns=x.columns[1])
x.rename(columns={'Crop year' : 'country'}, inplace=True)
x = x.melt(id_vars='country', var_name = 'crop_year', value_name='exports_1k_bags')
x['country'] = x['country'].str.strip()
x['exports_1k_bags'] = x['exports_1k_bags'].astype('float', errors='ignore')
x['exports_kg'] = x['exports_1k_bags'] * 1000 * 60
x['exports_lb'] = x['exports_kg'] * 2.20462262185
harvest_groups = ['April group', 'July group', 'October group']
totals = ['Total']
x['harvest_group'] = np.where(x['country'].isin(harvest_groups), x['country'], np.NaN)
x['harvest_group'] = x['harvest_group'].ffill()
hg_rows = x.iloc[:,0].isin(harvest_groups)
hg_df = x.loc[hg_rows]
tot_rows = x.iloc[:,0].isin(totals)
tot_df = x.loc[tot_rows]
x = x.loc[~(hg_rows|tot_rows)]
hg_df.drop(columns='country', inplace=True)
tot_df.drop(columns=['country', 'harvest_group'], inplace=True)
x = x.merge(hg_df, how = 'left', on = ['crop_year', 'harvest_group'], suffixes=['', '_hg'])
x = x.merge(tot_df, how = 'left', on = ['crop_year'], suffixes=['', '_total'])
x = x[['harvest_group', 'country', 'crop_year', 'exports_1k_bags', 'exports_kg', 'exports_lb', 
'exports_1k_bags_hg', 'exports_kg_hg', 'exports_lb_hg', 'exports_1k_bags_total', 'exports_kg_total', 'exports_lb_total']]
dfs['exports_crop_year'] = x

In [108]:
x = dfs['exports_calendar_year']
x.columns = x.iloc[2, :]
x = x.iloc[3:60, 0:32]
x = x.dropna(how='all')
x = x.drop(columns=x.columns[1])
x.rename(columns={'Calendar years' : 'country'}, inplace=True)
x = x.melt(id_vars='country', var_name = 'calendar_year', value_name='exports_1k_bags')
x['country'] = x['country'].str.strip()
x['exports_1k_bags'] = x['exports_1k_bags'].astype('float', errors='ignore')
x['exports_kg'] = x['exports_1k_bags'] * 1000 * 60
x['exports_lb'] = x['exports_kg'] * 2.20462262185
totals = ['Total']
tot_rows = x.iloc[:,0].isin(totals)
tot_df = x.loc[tot_rows]
x = x.loc[~(tot_rows)]
tot_df.drop(columns=['country'], inplace=True)
x = x.merge(tot_df, how = 'left', on = ['calendar_year'], suffixes=['', '_total'])
x = x[['country', 'calendar_year', 'exports_1k_bags', 'exports_kg', 'exports_lb',
 'exports_1k_bags_total', 'exports_kg_total', 'exports_lb_total']]
dfs['exports_calendar_year'] = x

In [109]:
x = dfs['imports']
x.columns = x.iloc[2, :]
x = x.iloc[3:43, 0:32]
x = x.dropna(how='all')
x = x.drop(columns=x.columns[1])
x.rename(columns={'Calendar years' : 'country'}, inplace=True)
x = x.melt(id_vars='country', var_name = 'calendar_year', value_name='imports_1k_bags')
x['country'] = x['country'].str.strip()
x['imports_1k_bags'] = x['imports_1k_bags'].astype('float', errors='ignore')
x['imports_kg'] = x['imports_1k_bags'] * 1000 * 60
x['imports_lb'] = x['imports_kg'] * 2.20462262185
eu = ['European Union']
eu_rows = x.iloc[:,0].isin(totals)
totals = ['Total']
tot_rows = x.iloc[:,0].isin(totals)
tot_df = x.loc[tot_rows]
x = x.loc[~(eu_rows|tot_rows)]
tot_df.drop(columns=['country'], inplace=True)
x = x.merge(tot_df, how = 'left', on = ['calendar_year'], suffixes=['', '_total'])
x = x[['country', 'calendar_year', 'imports_1k_bags', 'imports_kg', 'imports_lb',
 'imports_1k_bags_total', 'imports_kg_total', 'imports_lb_total']]
dfs['imports'] = x


In [110]:
x = dfs['re_exports']
x.columns = x.iloc[2, :]
x = x.iloc[3:43, 0:32]
x = x.dropna(how='all')
x = x.drop(columns=x.columns[1])
x.rename(columns={'Calendar years' : 'country'}, inplace=True)
x = x.melt(id_vars='country', var_name = 'calendar_year', value_name='re_exports_1k_bags')
x['country'] = x['country'].str.strip()
x['re_exports_1k_bags'] = x['re_exports_1k_bags'].astype('float', errors='ignore')
x['re_exports_kg'] = x['re_exports_1k_bags'] * 1000 * 60
x['re_exports_lb'] = x['re_exports_kg'] * 2.20462262185
eu = ['European Union']
eu_rows = x.iloc[:,0].isin(totals)
totals = ['Total']
tot_rows = x.iloc[:,0].isin(totals)
tot_df = x.loc[tot_rows]
x = x.loc[~(eu_rows|tot_rows)]
tot_df.drop(columns=['country'], inplace=True)
x = x.merge(tot_df, how = 'left', on = ['calendar_year'], suffixes=['', '_total'])
x = x[['country', 'calendar_year', 're_exports_1k_bags', 're_exports_kg', 're_exports_lb',
 're_exports_1k_bags_total', 're_exports_kg_total', 're_exports_lb_total']]
dfs['re_exports'] = x

In [111]:
x = dfs['prices_paid_to_growers']
x.columns = x.iloc[2, :]
x = x.iloc[3:76, 0:28]
x = x.dropna(how='all')
x = x.drop(columns=x.columns[1])
x.rename(columns={'Calendar years' : 'country'}, inplace=True)
x = x.melt(id_vars='country', var_name = 'calendar_year', value_name='price_paid_cents_per_lb')
x['country'] = x['country'].str.strip()
x['price_paid_cents_per_lb'] = x['price_paid_cents_per_lb'].astype('float', errors='ignore')
x['price_paid_dollars_per_lb'] = x['price_paid_cents_per_lb'] * 100
categories = ['Colombian Milds', 'Other Milds', 'Brazilian Naturals', 'Robustas']
totals = ['Total']
x['indicator_name'] = np.where(x['country'].isin(categories), x['country'], np.NaN)
x['indicator_name'] = x['indicator_name'].ffill()
cat_rows = x.iloc[:,0].isin(categories)
tot_rows = x.iloc[:,0].isin(totals)
x = x.loc[~(cat_rows|tot_rows)]
x = x[['country', 'indicator_name', 'calendar_year', 'price_paid_cents_per_lb', 'price_paid_dollars_per_lb']]
dfs['prices_paid_to_growers'] = x

In [112]:
x = dfs['indicator_prices']
x.columns = x.iloc[2, :]
x = x.iloc[3:424, 0:6]
x = x.dropna(how='all')
x.columns = x.columns.fillna('calendar_month').str.replace('\\n', '', regex=True).str.replace(' ', '_', regex=True).str.lower()
x = x.melt(id_vars='calendar_month', var_name='indicator_name', value_name='indicator_price_cents_per_lb')
x['indicator_price_cents_per_lb'] = x['indicator_price_cents_per_lb'].astype('float', errors = 'ignore')
x['indicator_price_dollars_per_lb'] = x['indicator_price_cents_per_lb'] * 100
year_rows = x['calendar_month'].astype('str').str.match(r'[0-9]{4}')
x['calendar_year'] = np.where(year_rows, x['calendar_month'], np.NaN)
x['calendar_year'] = x['calendar_year'].fillna(method='ffill')
avg_annual = x[year_rows].drop(columns='calendar_month')
x = x[~year_rows]
x = x.merge(avg_annual, how = 'left', on = ['calendar_year', 'indicator_name'], suffixes=['', '_avg'])
x = x[['calendar_year','calendar_month','indicator_name','indicator_price_cents_per_lb','indicator_price_dollars_per_lb', 'indicator_price_cents_per_lb_avg','indicator_price_dollars_per_lb_avg']]
dfs['indicator_prices'] = x

In [113]:
x = dfs['non_member_imports']
x.columns = x.iloc[2, :]
x = x.iloc[3:129, 0:32]
x = x.dropna(how='all')
x = x.drop(columns=x.columns[1])
x.rename(columns={'Calendar years' : 'country'}, inplace=True)
x = x.melt(id_vars='country', var_name = 'calendar_year', value_name='imports_1k_bags')
x['country'] = x['country'].str.strip()
x['imports_1k_bags'] = x['imports_1k_bags'].astype('float', errors='ignore')
x['imports_kg'] = x['imports_1k_bags'] * 1000 * 60
x['imports_lb'] = x['imports_kg'] * 2.20462262185
china_agg = x.iloc[:,0].isin(["China, People's Republic of"])
x = x[~china_agg]
regions = ['Africa','Asia & Oceania','Caribbean','Central America & Mexico','Europe','North America','South America']
totals = ['Total']
x['region'] = np.where(x['country'].isin(regions), x['country'], np.NaN)
x['region'] = x['region'].ffill()
region_rows = x.iloc[:,0].isin(regions)
region_df = x.loc[region_rows]
tot_rows = x.iloc[:,0].isin(totals)
tot_df = x.loc[tot_rows]
x = x.loc[~(region_rows|tot_rows)]
region_df.drop(columns='country', inplace=True)
tot_df.drop(columns=['country', 'region'], inplace=True)
x = x.merge(region_df, how = 'left', on = ['calendar_year', 'region'], suffixes=['', '_regional'])
x = x.merge(tot_df, how = 'left', on = ['calendar_year'], suffixes=['', '_total'])
x = x[['region','country','calendar_year','imports_1k_bags','imports_kg','imports_lb','imports_1k_bags_regional',
'imports_kg_regional','imports_lb_regional','imports_1k_bags_total','imports_kg_total','imports_lb_total']]
dfs['non_member_imports'] = x

In [ ]:
non_member_re_exports

In [127]:
x = dfs['non_member_re_exports']
x.columns = x.iloc[2, :]
x = x.iloc[3:129, 0:32]
x = x.dropna(how='all')
x = x.drop(columns=x.columns[1])
x.rename(columns={'Calendar years' : 'country'}, inplace=True)
x = x.melt(id_vars='country', var_name = 'calendar_year', value_name='re_exports_1k_bags')
x['country'] = x['country'].str.strip()
x['re_exports_1k_bags'] = x['re_exports_1k_bags'].astype('float', errors='ignore')
x['re_exports_kg'] = x['re_exports_1k_bags'] * 1000 * 60
x['re_exports_lb'] = x['re_exports_kg'] * 2.20462262185
china_agg = x.iloc[:,0].isin(["China, People's Republic of"])
x = x[~china_agg]
regions = ['Africa','Asia & Oceania','Caribbean','Central America & Mexico','Europe','North America','South America']
totals = ['Total']
x['region'] = np.where(x['country'].isin(regions), x['country'], np.NaN)
x['region'] = x['region'].ffill()
region_rows = x.iloc[:,0].isin(regions)
region_df = x.loc[region_rows]
tot_rows = x.iloc[:,0].isin(totals)
tot_df = x.loc[tot_rows]
x = x.loc[~(region_rows|tot_rows)]
region_df.drop(columns='country', inplace=True)
tot_df.drop(columns=['country', 'region'], inplace=True)
x = x.merge(region_df, how = 'left', on = ['calendar_year', 'region'], suffixes=['', '_regional'])
x = x.merge(tot_df, how = 'left', on = ['calendar_year'], suffixes=['', '_total'])
x = x[['region','country','calendar_year','re_exports_1k_bags','re_exports_kg','re_exports_lb','re_exports_1k_bags_regional',
're_exports_kg_regional','re_exports_lb_regional','re_exports_1k_bags_total','re_exports_kg_total','re_exports_lb_total']]
dfs['non_member_re_exports'] = x

In [128]:
dfs.keys()

dict_keys(['total_production', 'domestic_consumption', 'gross_opening_stocks', 'exports_crop_year', 'exports_calendar_year', 'imports', 're_exports', 'prices_paid_to_growers', 'retail_prices', 'indicator_prices', 'non_member_imports', 'non_member_re_exports'])

In [132]:
for k in dfs.keys():
    dfs[source_files.loc[i,'variable_type']].to_csv(f'{interim_dir}/{k}.csv', index=False)